# Imports #

In [7]:
import pandas as pd
import numpy as np
import datetime
import time
from decouple import config
import requests
import urllib.parse
import json
AAapikey = config('AAKey')

# Gather Historic Stock News #

In [8]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)

In [9]:
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1), interval = 365):
    # Format for news is [Date, Headline, Ticker]
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    url = 'https://www.alphavantage.co/query?'
    window = 15
    startDate = endDate - datetime.timedelta(days = interval)
    windowStart = startDate
    windowEnd = startDate + datetime.timedelta(days = window)
    delta = datetime.timedelta(days = window)
    count = 0
    while (windowEnd <= endDate):
        if count == 5:
            time.sleep(60)
            count = 0
        startDateAA = toAADate(windowStart)
        endDateAA = toAADate(windowEnd)
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDateAA, 'time_to': endDateAA, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
        r = requests.get(url, params = Myparams)
        data = r.json()
        try:
            for i in data.get("feed"):
                test_date = i.get("time_published")
                test_date = test_date[:8]
                newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date() 
                row = [newDate, i.get("title"), ticker]
                new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
                historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
        except:
            print("Request failed")
            print("Starting date: " +str(windowStart))
            print("Starting date: " +str(windowEnd))
            print(data)
            break
        windowStart = windowEnd
        windowEnd = windowEnd + delta
        count = count + 1
    print("Final rolling date is: " +str(windowEnd))
    #print(historic_parsed_news)
    return historic_news


In [10]:

ticker = input('Input Ticker:\n')  
print(ticker)

MSFT


In [11]:
hist = getHistoricNewsData(ticker)

Final rolling date is: 2023-03-02


In [12]:
print(hist.head)

<bound method NDFrame.head of             Date                                           Headline Ticker
0     2022-03-01  MSFT: Better Buy for 2022: Salesforce.com vs. ...   MSFT
1     2022-03-02  Facebook, YouTube, TikTok restrict access to R...   MSFT
2     2022-03-02  Apple, Google, Netflix, and Visa join the grow...   MSFT
3     2022-03-02  Ukraine calls on gaming industry to suspend bu...   MSFT
4     2022-03-02  TikTok faces delicate dance to keep all sides ...   MSFT
...          ...                                                ...    ...
4623  2023-02-02  How to protect yourself from tax fraud and sca...   MSFT
4624  2023-02-02  Zacks Earnings Trends Highlights: Microsoft, I...   MSFT
4625  2023-02-02  Pinterest Bows Down To Macro Uncertainties; Le...   MSFT
4626  2023-02-02  Samsung Hints Metaverse Ambitions At Product L...   MSFT
4627  2023-02-02      US filings for jobless aid lowest since April   MSFT

[4628 rows x 3 columns]>


In [13]:
histname = "datasets/" + ticker + "-365-day-hist.csv"
hist.to_csv(histname)